# Migrate data from a backup to this repository

Data was manually backed up from http://discovery.dartmouth.edu/model_free_data/ on 2014-04-01. Migrate this locally-stored backup to this repository. Data files are modernized.

In [1]:
import os
import shutil

In [2]:
names = [
    'threeway',
    'HWthreeway',
    'fourway',
    'HWfourway',
    'fourwayNoLow',
    'fiveway',
    'HWfiveway',
    'fivewayNoLow',
]

## Best datasets

In [ ]:
for name in names:
    best_dir = os.path.join('data', name, 'best-datasets')
    ! mkdir --parents {best_dir} && \
      tar --extract --gunzip --directory {best_dir} --file legacy/{name}Bests.tar.gz --strip-components=1 && \
      cd {best_dir} && \
      rename -force 's/.txt$/.tsv/' *.txt && \
      sed --in-place '1s/^/Set_Number\t/' dataInfo.tsv && \
      mv dataInfo.tsv ../best-attributes.tsv

## All datasets

In [ ]:
for name in names:
    src = os.path.join('legacy', name + 'Compilation.txt')
    dst = os.path.join('data', name, 'all-attributes.tsv')
    shutil.copy(src, dst)
    ! sed --in-place 's/"//g' {dst} # Remove quotes

### Parse the compressed tarballs

In [ ]:
root = os.getcwd()
for name in names:
    print(name)
    path = os.path.join(root, 'legacy', 'set-{}.tar.gz'.format(name))
    all_dir = os.path.join(root, 'data', name, 'all-datasets')
    ! rm --recursive --force {all_dir} && \
      mkdir --parents {all_dir}
    if 'NoLow' in name:
        ! tar --extract --gunzip --directory {all_dir} --file {path} --strip-components=1
        ! rm --recursive {all_dir}/{name}Bests
    else:
        ! tar --extract --gunzip --directory {all_dir} --file {path}
    set_ids = [x for x in os.listdir(all_dir) if x.startswith('set')]
    for set_id in set_ids:
        set_dir = os.path.join(all_dir, set_id)
        ! cd {set_dir} && \
          rename -force 's/.txt$/.tsv/' *.txt && \
          bzip2 tracker.tsv && \
          rm thaBest.tsv && \
          mv picker.tsv attributes.tsv && \
          sed --in-place 's/"//g' attributes.tsv && \
          tar --create --bzip2 --file datasets.tar.bz2 evolved* --remove-files